In [ ]:
import itertools as it
import operator
import re
import subprocess
import tempfile
import time
from collections import Counter
from pathlib import Path

import awkward as ak
import bottleneck as bn
import duckdb
import gfapy
import holoviews as hv
import ibis
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
import polars as pl
import pyabpoa
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pyfastx
import pysam
import spoa
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from pyarrow import csv
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.consensus as con
import paulssonlab.sequencing.io as sio
import paulssonlab.sequencing.processing as processing
from paulssonlab.util.sequence import reverse_complement

In [ ]:
hv.extension("bokeh")

In [ ]:
%load_ext pyinstrument
import line_profiler
import pyinstrument

%load_ext line_profiler

In [ ]:
pl.enable_string_cache()

# Consensus

In [ ]:
%%time
df = pl.scan_ipc(
    "/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/230707_repressilators/dorado_0.4.0/uncompressed/prepared/*.arrow"
)
res = (
    (
        processing.compute_depth(df.filter(pl.col("path").hash() % 1000 == 4))
        .filter(pl.col("duplex_depth") > 10)
        .select(
            pl.col(
                "name",
                "is_duplex",
                "read_seq",
                "read_phred",
                "reverse_complement",
                "path",
                "depth",
                "simplex_depth",
                "duplex_depth",
            )
        )
        .group_by("path")
        .agg(
            # pl.col("name", "is_duplex", "read_seq", "read_phred", "reverse_complement"),
            pl.col(
                "name", "is_duplex", "read_seq", "read_phred", "reverse_complement"
            ).map_elements(partial(con.get_consensus_df, use_phreds=False)),
            pl.col("path", "depth", "simplex_depth", "duplex_depth").first(),
        )
    )
    .sort("depth", descending=True)
    .collect()
)
# res.select(pl.all().exclude("name","read_seq", "read_phred", "reverse_complement"))

In [ ]:
test_reads = res[
    0
]  # .explode(pl.col("name", "read_seq", "read_phred", "reverse_complement", "is_duplex"))

In [ ]:
test_reads.head()

In [ ]:
ground_truth = consensus_func(test_reads.head())

In [ ]:
seqs, phreds = con.prepare_reads(
    test_reads.get_column("read_seq").to_list(),
    test_reads.get_column("reverse_complement").to_arrow(),
    test_reads.get_column("read_phred").to_arrow(),
)

In [ ]:
%%time
# con_seqs = con.poa(seqs, method="abpoa", num_consensus=1, sort=True)
con_seqs, msa_seqs = con.poa(
    seqs, method="spoa", num_consensus=1, return_msa=False, sort=True
)

In [ ]:
len(con_seqs)

In [ ]:
len(msa_seqs)

In [ ]:
%%time
df = pl.scan_ipc(
    "/home/jqs1/scratch/jqs1/sequencing/230930_alignment_test/230707_repressilators/dorado_0.4.0/uncompressed/prepared/*.arrow"
)
res = (
    processing.compute_depth(df.filter(pl.col("path").hash() % 1000 == 4))
    .filter(pl.col("duplex_depth") > 10)
    .select(
        pl.col(
            "name",
            "is_duplex",
            "read_seq",
            "read_phred",
            "reverse_complement",
            "path",
        )
    )
    .group_by("path")
    .map_groups(
        consensus_func,
        schema=dict(
            path_subset=pl.List(pl.Categorical),
            read_seq=pl.Utf8,
            read_phred=pl.Utf8,
            depth_simplex=pl.UInt64,
            depth_duplex=pl.UInt64,
        ),
    )
).collect()
# res.select(pl.all().exclude("name","read_seq", "read_phred", "reverse_complement"))

In [ ]:
res

# Old consensus

In [ ]:
group_depths = (
    test_groups.select(pl.col("name"))
    .with_columns(pl.col("name").list.len())
    .filter(pl.col("name") > 1)
    .sort("name", descending=True)
    .to_numpy()
)

In [ ]:
plt.plot(group_depths[:, 0])

In [ ]:
%%time
test_groups_subset = test_groups.filter(pl.col("name").list.len().is_between(100, 120))

In [ ]:
test_groups_subset.select("depth", "duplex_depth")

In [ ]:
test_reads = (
    test_groups_subset[0]
    .select(
        pl.col("name", "read_seq", "read_phred", "reverse_complement").list.explode(),
        pl.col("duplex_depth", "simplex_depth"),
    )
    .with_columns(
        pl.col("read_seq").str.len_bytes().alias("read_len"),
        pl.col("name").str.contains(";").alias("is_duplex"),
        pl.col("name").str.contains(";").not_().alias("is_simplex"),
    )
    .sort("is_duplex", descending=True)
)
test_reads

In [ ]:
plt.hist(test_reads.get_column("read_len"), bins=100);

In [ ]:
test_reads = test_reads.filter(pl.col("read_len").is_between(3300, 3700))

In [ ]:
%%time
seqs, phreds = con.prepare_reads(
    test_reads.get_column("read_seq").to_list(),
    test_reads.get_column("reverse_complement").to_arrow(),
    test_reads.get_column("read_phred").to_arrow(),
)

In [ ]:
%%time
msa_seqs = con.msa(seqs, method="abpoa", aln_mode="l")

In [ ]:
seq_global = con.chars_to_str(consensus_global[0])
seq_local = con.chars_to_str(consensus_local[0])
(len(seq_global), len(seq_local))

In [ ]:
print(seq_global)

In [ ]:
print(seq_local)

In [ ]:
con.print_msa(msa_seqs, phreds)

# GraphAligner

In [ ]:
def run_aligner(gfa_filename, reads_filename, args=[]):
    cmd_base = ["/home/jqs1/paulsson-home/bin/GraphAligner"]
    with tempfile.NamedTemporaryFile(mode="w+", suffix=".gaf") as gaf_file:
        cmd = [
            *cmd_base,
            "-g",
            gfa_filename,
            "-f",
            reads_filename,
            "-a",
            gaf_file.name,
            *args,
        ]
        start = time.time()
        out = subprocess.run(cmd, capture_output=True)
        stop = time.time()
        if out.returncode != 0:
            print("STDOUT:")
            print(out.stdout.decode())
            print()
            print("STDERR:")
            print(out.stderr.decode())
            print()
            raise RuntimeError("GraphAligner returned non-zero exit status")
        runtime = stop - start
        # print("STDOUT")
        # print(out.stdout.decode())
        # print("STDERR")
        # print(out.stderr.decode())
        gaf = sio.read_gaf(gaf_file.name)
        return gaf, runtime


def align_reads(gfa_filename, reads, args=["-x", "vg"]):
    with tempfile.NamedTemporaryFile(mode="w+", suffix=".fasta") as reads_file:
        formatted_reads = (
            "\n".join([f">r{idx}\n{read}" for idx, read in enumerate(reads)]) + "\n"
        )
        reads_file.write(formatted_reads)
        reads_file.flush()
        return run_aligner(gfa_filename, reads_file.name, args=args)[0]

## Read group alignment

In [ ]:
test_groups_subset[0]

In [ ]:
test_reads = (
    test_groups_subset[idx]
    .select(
        pl.col("name", "read_seq", "read_phred", "reverse_complement").list.explode(),
        pl.col("duplex_depth", "simplex_depth"),
    )
    .with_columns(
        pl.col("read_seq").str.len_bytes().alias("read_len"),
        pl.col("name").str.contains(";").alias("is_duplex"),
        pl.col("name").str.contains(";").not_().alias("is_simplex"),
    )
    .sort("is_duplex", descending=True)
)

In [ ]:
%%time
seqs, phreds = con.prepare_reads(
    test_reads.get_column("read_seq").to_list(),
    test_reads.get_column("reverse_complement").to_arrow(),
    test_reads.get_column("read_phred").to_arrow(),
)

In [ ]:
%%time
msa_seqs = con.msa(seqs[:5], method="abpoa", aln_mode="g")
(
    consensus_seq,
    consensus_phred,
    nonconsensus_seq,
    nonconsensus_phred,
) = con.phred_weighted_consensus(msa_seqs, phreds)

In [ ]:
%%time
gaf = align_reads(gfa_filename, seqs, args=["-x", "dbg"])

In [ ]:
gaf["path_length"]

In [ ]:
plt.hist(gaf["NM"], bins=100);

In [ ]:
aligner = pyabpoa.msa_aligner(aln_mode="g")
res = aligner.msa(seqs[:10], out_cons=True, out_msa=True)
# msa_seqs = res.msa_seq

In [ ]:
con_seq, msa_seqs = spoa.poa(seqs[:10])

In [ ]:
print(con_seq)

In [ ]:
print(res.cons_seq[0])

In [ ]:
print(seqs[1])

In [ ]:
res.msa_seq[0]

In [ ]:
print(con.chars_to_str(consensus_seq))

In [ ]:
print(con.chars_to_str(msa_seqs[3], True))

In [ ]:
gaf["cg"]

In [ ]:
gaf.column_names

In [ ]:
gaf["name"]

In [ ]:
print(seqs[0])

In [ ]:
print("\n".join([f">r{idx}\n{seq}" for idx, seq in enumerate(seqs)]) + "\n")

## Consensus alignment

In [ ]:
%%time
res = []
for idx in trange(len(test_groups_subset)):
    test_reads = (
        test_groups_subset[idx]
        .select(
            pl.col(
                "name", "read_seq", "read_phred", "reverse_complement"
            ).list.explode(),
            pl.col("duplex_depth", "simplex_depth"),
        )
        .with_columns(
            pl.col("read_seq").str.len_bytes().alias("read_len"),
            pl.col("name").str.contains(";").alias("is_duplex"),
            pl.col("name").str.contains(";").not_().alias("is_simplex"),
        )
        .sort("is_duplex", descending=True)
    )
    seqs, phreds = con.prepare_reads(
        test_reads.get_column("read_seq").to_list(),
        test_reads.get_column("reverse_complement").to_arrow(),
        test_reads.get_column("read_phred").to_arrow(),
    )
    msa_seqs = con.msa(seqs, method="abpoa", aln_mode="l")
    (
        consensus_seq,
        consensus_phred,
        nonconsensus_seq,
        nonconsensus_phred,
    ) = con.phred_weighted_consensus(msa_seqs, phreds)
    res.append(
        dict(
            consensus_seq=consensus_seq,
            consensus_phred=consensus_phred,
            nonconsensus_seq=nonconsensus_seq,
            nonconsensus_phred=nonconsensus_phred,
        )
    )

In [ ]:
%%time
gaf = align_reads(
    gfa_filename, [con.chars_to_str(r["consensus_seq"]) for r in res], args=["-x", "vg"]
)

In [ ]:
gaf["name"]

In [ ]:
gaf